- Utilisation de bigml pour créer des modèles d'apprentissage

In [5]:
from pandas import DataFrame, read_csv
from bigml.api import BigML
api = BigML(project="project/5d94a42042129f2b0500005c")

- Importation des dataframe depuis les fichiers générés précédemment

In [6]:
df = read_csv('fulltrain_mod.csv')
df2 = read_csv('test_mod.csv')

- Création des sources et datasets correspondants du fichier trainfull sur le site bigml

In [7]:
source_args = {"name": "171007 fulltrain ipnb",
     "source_parser": {"missing_tokens": ["NULL"]}}
source = api.create_source('fulltrain_mod.csv', source_args)

dataset_args = {"name": "171007 ipnb trainfull"}
dataset = api.create_dataset(source, dataset_args)

- Création du split trainfull en 80/20 pour train/test

In [22]:
train_dataset = api.create_dataset(
    dataset, {"name": "171007 ipnb - Train",
                     "sample_rate": 0.8, "seed": "my_seed"})

val_dataset = api.create_dataset(
    dataset, {"name": "171007 ipnb - Validation",
                     "sample_rate": 0.8, "seed": "my_seed", "out_of_bag": True})

- Génération du modèle d'apprentissage

In [23]:
model_args = {"objective_field": "SeriousDlqin2yrs"}
model = api.create_ensemble(train_dataset, model_args)

In [24]:
api.ok(model)

True

In [25]:
model = api.get_ensemble(model)

In [22]:
api.pprint(model['object']['ensemble'])

{   'fields': {   '000000': {   'column_number': 0,
                                'datatype': 'int32',
                                'name': 'field1',
                                'optype': 'numeric',
                                'order': 0,
                                'preferred': True,
                                'summary': {   'bins': [   [2555.1253, 4110],
                                                           [6217.1521, 1762],
                                                           [9636.38334, 3793],
                                                           [14931.72794, 4760],
                                                           [19728.46324, 2938],
                                                           [23621.01454, 3301],
                                                           [28150.11395, 3914],
                                                           [33832.19359, 5088],
                                                           [38

                                               'minimum': 0,
                                               'missing_count': 0,
                                               'population': 120000,
                                               'skewness': 23.23448,
                                               'standard_deviation': 12.48928,
                                               'sum': 111635,
                                               'sum_squares': 18821553,
                                               'variance': 155.98213}},
                  '00000f': {   'column_number': 15,
                                'datatype': 'double',
                                'name': 'IncomePerPerson',
                                'optype': 'numeric',
                                'order': 15,
                                'preferred': True,
                                'summary': {   'bins': [   [3952.71798, 93492],
                                                      

- Evaluation du modèle (AUC, matrice de confusion, ...)

In [21]:
evaluation = api.create_evaluation(model, val_dataset, {
    "name": "171007 ipnb - Train - evaluation"})

- Récupérer des données existantes sur bigml

In [28]:
model = api.get_ensemble("ensemble/5db162ae7811dd0557000e0e")
val_dataset = api.get_dataset("dataset/5db162955299631c82008bd0")
evaluation = api.get_evaluation("evaluation/5db162d07811dd7f2d00872a")

- Matrice de confusion

In [34]:
api.pprint(evaluation['object']['result']['model']['confusion_matrix'])

[[27739, 223], [1711, 327]]


- Critère de performance ROC AUC

In [38]:
api.pprint(evaluation['object']['result']['model']['per_class_statistics'][0]['area_under_roc_curve'])

0.85905


- Génération de la prédiction sur le 20% validation

In [41]:
prediction_args = {"name": "171007 ipnb - Train (80%) - 20% Prediction",
                  "header": True,
                  "probabilities": True,
                  "all_fields": True}

batch_prediction = api.create_batch_prediction(model, val_dataset, prediction_args)

In [42]:
api.ok(batch_prediction)

True

- Téléchargement du fichier de prédiction

In [43]:
api.download_batch_prediction(batch_prediction,
                              filename= "171007 ipnb 80% - val prediction.csv")

'171007 ipnb 80% - val prediction.csv'

In [86]:
df4 = read_csv('171007 ipnb 80% - val prediction.csv')
df4

,field1,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,MonthlyIncome_sansna,Sum,IncomePerPerson,SeriousDlqin2yrs.1,0 probability,1 probability
0,9,10,0,0.189169,57,0,0.606291,23684.0,9,0,4,0,2.0,23684,0,7894.666667,0,0.97277,0.02723
1,11,12,0,0.018798,51,0,0.531529,6501.0,7,0,2,0,2.0,6501,0,2167.000000,0,0.98023,0.01977
2,20,21,0,0.200923,43,0,0.430046,12300.0,10,0,2,0,0.0,12300,0,12300.000000,0,0.98885,0.01115
3,42,43,0,0.469057,31,0,0.048211,9250.0,4,0,0,0,1.0,9250,0,4625.000000,0,0.96847,0.03153
4,49,50,0,0.000080,70,0,0.256340,6900.0,21,1,1,0,0.0,6900,1,6900.000000,0,0.85043,0.14957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,149977,149978,0,0.236450,29,0,349.000000,NaN,3,0,0,0,0.0,0,0,NaN,0,0.98429,0.01571
29996,149982,149983,0,0.021046,37,0,0.250272,2760.0,8,0,0,0,3.0,2760,0,690.000000,0,0.98963,0.01037
29997,149986,149987,0,0.168102,49,0,0.080384,5000.0,16,0,0,0,1.0,5000,0,2500.000000,0,0.96253,0.03747
29998,149988,149989,0,0.902051,31,1,0.347924,7515.0,10,0,1,0,0.0,7515,1,7515.000000,0,0.87037,0.12963


- Calcul matrice de confusion à la main

In [112]:
Preal = df4['SeriousDlqin2yrs'] == 1
Nreal = df4['SeriousDlqin2yrs'] == 0
Ppred = df4['SeriousDlqin2yrs.1'] == 1
Npred = df4['SeriousDlqin2yrs.1'] == 0
df4.loc[Ppred & Preal, 'Eval'] = 'TP'
df4.loc[Ppred & Nreal, 'Eval'] = 'FP'
df4.loc[Npred & Nreal, 'Eval'] = 'TN'
df4.loc[Npred & Preal, 'Eval'] = 'FN'
df4

,field1,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,MonthlyIncome_sansna,Sum,IncomePerPerson,SeriousDlqin2yrs.1,0 probability,1 probability,Eval
0,9,10,0,0.189169,57,0,0.606291,23684.0,9,0,4,0,2.0,23684,0,7894.666667,0,0.97277,0.02723,TN
1,11,12,0,0.018798,51,0,0.531529,6501.0,7,0,2,0,2.0,6501,0,2167.000000,0,0.98023,0.01977,TN
2,20,21,0,0.200923,43,0,0.430046,12300.0,10,0,2,0,0.0,12300,0,12300.000000,0,0.98885,0.01115,TN
3,42,43,0,0.469057,31,0,0.048211,9250.0,4,0,0,0,1.0,9250,0,4625.000000,0,0.96847,0.03153,TN
4,49,50,0,0.000080,70,0,0.256340,6900.0,21,1,1,0,0.0,6900,1,6900.000000,0,0.85043,0.14957,TN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,149977,149978,0,0.236450,29,0,349.000000,NaN,3,0,0,0,0.0,0,0,NaN,0,0.98429,0.01571,TN
29996,149982,149983,0,0.021046,37,0,0.250272,2760.0,8,0,0,0,3.0,2760,0,690.000000,0,0.98963,0.01037,TN
29997,149986,149987,0,0.168102,49,0,0.080384,5000.0,16,0,0,0,1.0,5000,0,2500.000000,0,0.96253,0.03747,TN
29998,149988,149989,0,0.902051,31,1,0.347924,7515.0,10,0,1,0,0.0,7515,1,7515.000000,0,0.87037,0.12963,TN


In [124]:
df4[['field1','Eval']].groupby('Eval').count()

,field1
Eval,
FN,1711
FP,223
TN,27739
TP,327


In [134]:
df4['bigerror'] = (df4['1 probability'] - df4['SeriousDlqin2yrs'])
df4.sort_values('bigerror', ascending=False)[:10]

,field1,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,...,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,MonthlyIncome_sansna,Sum,IncomePerPerson,SeriousDlqin2yrs.1,0 probability,1 probability,Eval,bigerror
23063,116003,116004,0,1.680710,51,1,0.178730,3384.0,7,2,...,3,1.0,3384,6,1692.0,1,0.23892,0.76108,FP,0.76108
8711,43704,43705,0,1.455090,35,2,0.065604,3200.0,3,6,...,1,1.0,3200,9,1600.0,1,0.26332,0.73668,FP,0.73668
3626,18633,18634,0,1.671105,52,3,0.513251,4527.0,8,3,...,1,4.0,4527,7,905.4,1,0.26670,0.73330,FP,0.73330
17902,89939,89940,0,0.874335,49,4,2146.000000,NaN,9,4,...,0,0.0,0,8,NaN,1,0.27535,0.72465,FP,0.72465
29838,149206,149207,0,0.465183,51,3,2824.000000,NaN,7,2,...,4,0.0,0,9,NaN,1,0.27698,0.72303,FP,0.72303
5390,27476,27477,0,1.000000,72,98,0.004888,4500.0,0,98,...,98,0.0,4500,294,4500.0,1,0.27867,0.72133,FP,0.72133
26256,131688,131689,0,1.212625,36,1,0.356739,3004.0,5,4,...,2,1.0,3004,7,1502.0,1,0.27884,0.72116,FP,0.72116
23887,120049,120050,0,1.000000,46,96,0.051765,4616.0,0,96,...,96,0.0,4616,288,4616.0,1,0.27894,0.72106,FP,0.72106
24201,121572,121573,0,1.389805,44,5,126.000000,NaN,2,3,...,1,0.0,0,9,NaN,1,0.28080,0.71920,FP,0.71920
28999,145149,145150,0,1.000000,36,3,3539.000000,NaN,4,2,...,1,1.0,0,6,NaN,1,0.28223,0.71778,FP,0.71778
